### Como manipular XML com XPath

In [4]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018-small.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)
    

### Monta Dicionario

In [5]:
4from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [13]:
wordModel = KeyedVectors.load_word2vec_format('health_w2v_unigram_50.bin', binary=True)

dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    qualifier = '#'
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text)
                palavra_similar = wordModel.most_similar_cosmul(sem_assento.lower(),topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.9 and qualifier != 'pharmacology'):
                        terms.append("<i>"+similar+"</i>" + " <input class='radio' type='radio' name='"+similar+"' value='1'/> Certo <input class='radio' type='radio' name='"+similar+"'' value='0'/> Errado")
                    elif(porcentagem > 0.95 and qualifier == 'pharmacology'):
                        terms.append("<i>"+similar+"</i>")
    
    terms.append(" <input type='hidden' name='ID' value='"+d.find('.DescriptorUI').text+"'/> ")                         
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': d.find('.DescriptorName/String').text,
        'scope': scope,
        'terms': sorted(set(terms), reverse=True),
        'qualifier': qualifier
    }
                
    #break
len(dictMesh)

1859

### Imprime Exemplo

In [14]:
sorted(terms)

[" <input type='hidden' name='ID' value='D000347'/> ",
 'Afibrinogenemia',
 'Afibrinogenemia',
 'Afibrinogenemias',
 'Deficiency, Fibrinogen',
 'Deficiência de Fibrinogênio',
 'Fibrinogen Deficiencies',
 'Fibrinogen Deficiency']

### Salva Dicionario

In [15]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [59]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()


In [30]:
evolucao = 'Wellcome 248U'
evolucao = evolucao.split(' ')
for i, palavra in enumerate(evolucao):

    ## Busca palavra no Mesh
    for dui in dictMesh:
        d = dictMesh[dui]
        for t in d['terms']:
            new_t = t.replace('<i>', '')
            new_t = new_t.replace('</i>', '')

            if i+1 < (len(evolucao)) and palavra.lower()+" "+evolucao[i+1].lower() == new_t.lower():
                    teste = dictMesh[dui]['terms']
                    termos = '<br/>- '.join(teste)
                    print(termos)
                    break
            elif new_t.lower() == palavra.lower():
                teste = dictMesh[dui]['terms']
                termos = '<br/>- '.join(teste)

9-((2-Hydroxyethoxy)methyl)guanine<br/>- Aci Sanorania<br/>- Aci-Sanorania<br/>- Acic<br/>- Aciclobeta<br/>- Acicloguanosina<br/>- Aciclostad<br/>- Aciclovir<br/>- Aciclovir<br/>- Aciclovir Alonga<br/>- Aciclovir Sanorania<br/>- Aciclovir-Sanorania<br/>- Acifur<br/>- Acipen Solutab<br/>- Acivir<br/>- Activir<br/>- Acyclo V<br/>- Acyclo-V<br/>- Acycloguanosine<br/>- Acyclovir<br/>- Acyclovir Sodium<br/>- Alonga, Aciclovir<br/>- Antiherpes Creme<br/>- Avirax<br/>- Cicloferon<br/>- Clonorax<br/>- Cusiviral<br/>- Genvir<br/>- Herpetad<br/>- Herpofug<br/>- Herpotern<br/>- Herpoviric<br/>- Isavir<br/>- Laciken<br/>- Mapox<br/>- Maynar<br/>- Milavir<br/>- Opthavir<br/>- Sodium, Acyclovir<br/>- Solutab, Acipen<br/>- Supraviran<br/>- Viclovir<br/>- Vipral<br/>- Virax Puren<br/>- Virax-Puren<br/>- ViraxPuren<br/>- Virherpes<br/>- Virmen<br/>- Virolex<br/>- Virupos<br/>- Virzin<br/>- Wellcome 248U<br/>- Wellcome-248U<br/>- Wellcome248U<br/>- Zoliparin<br/>- Zovirax<br/>- Zyclir<br/>- aciclovir vo

In [27]:
termos=" "
confirma = False
evolucao = "abd"
strr = ""

#Verifica a lista para ver se a palavra esta no dicionario
cont = 0
for palavra in evolucao:
    for dui in dictMesh:
        d = dictMesh[dui]
        for t in d['terms']:
            if t.lower() == palavra.lower():
                evolucao[cont] = '<a href="#" data-ui="das" data-term="" data-scope="'+d['scope']+'">'+palavra+'</a>'
                cont +=1

    teste = dictMesh[dui]['terms']
    termos = ' '.join(teste)
evolucao = evolucao.replace('data-terms=""', 'data-terms="'+termos+'"')#pegar os termos da lista e por aqui
strr += ' '+evolucao

#print(termos)
termos

'Fibrinogen Deficiency Deficiência de Fibrinogênio Fibrinogen Deficiencies Afibrinogenemia Afibrinogenemias Deficiency, Fibrinogen'

In [14]:
'''dictValida = {}
import gzip, pickle
with gzip.open('dictValida.dict.gz','wb') as fp:
    pickle.dump(dictValida,fp)
    fp.close()'''

In [1]:
import gzip, pickle
with gzip.open("dictValida.dict.gz",'rb') as fd:
    dictValida = pickle.load(fd)
    fd.close()

In [2]:
dictValida

{'teria': {'ID': 'D014005', 'target': '0'},
 'corporis': {'ID': 'D014005', 'target': '0'},
 'tem': {'ID': 'D014005', 'target': '0'},
 'teriam': {'ID': 'D014005', 'target': '0'},
 'possuia': {'ID': 'D014005', 'target': '0'},
 'encontrou': {'ID': 'D014005', 'target': '0'},
 'pegou': {'ID': 'D014005', 'target': '0'},
 'tinham': {'ID': 'D014005', 'target': '0'},
 'b351': {'ID': 'D014005', 'target': '0'},
 'ca': {'ID': 'D001539', 'target': '0'},
 'neoplas': {'ID': 'D001539', 'target': '0'},
 'neoplaisa': {'ID': 'D001539', 'target': '0'},
 'estomago': {'ID': 'D004383', 'target': '0'},
 'febricula': {'ID': 'D005334', 'target': '0'},
 'hipertemia': {'ID': 'D005334', 'target': '0'},
 'porem': {'ID': 'D031668', 'target': '0'},
 'ainda': {'ID': 'D031668', 'target': '0'},
 'muito': {'ID': 'D031668', 'target': '0'},
 '11tc': {'ID': 'D000072078', 'target': '0'},
 '12tc': {'ID': 'D000072078', 'target': '0'},
 'tc': {'ID': 'D000072078', 'target': '1'},
 '15tc': {'ID': 'D000072078', 'target': '0'},
 '2

In [4]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
cont_adicionados = 0
cont_def_enriquecidas = 0
cont_def = 0
cont_anatomy = 0 
cont_methods=0
cont_diagnosis=0
cont_pharmacology=0
cont_other=0
cont_anatomy_total = 0 
cont_methods_total =0
cont_diagnosis_total =0
cont_pharmacology_total =0
cont_other_total =0
cont_originais=0
cont_certo_valida=0
cont_certo_anatomy = 0 
cont_certo_methods=0
cont_certo_diagnosis=0
cont_certo_pharmacology=0
cont_certo_other=0
cont_avaliados = 0

#Termos novos adicionados

for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if('<i>' in t):
            cont_adicionados +=1
            
print("Termos novos adicionados: "+str(cont_adicionados))

#Definicoes enriquecidas
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if('<i>' in t):
            cont_def_enriquecidas +=1
            break

print("\nDefinicoes enriquecidas: "+str(cont_def_enriquecidas))

#Definicoes
for dui in dictMesh:
    cont_def +=1
    
print("Definicoes: "+str(cont_def))

"""#Definicoes por categoria
for dui in dictMesh:
    d = dictMesh[dui]
    if(d['qualifier'] == 'anatomy & histology'):
        cont_anatomy +=1
    elif(d['qualifier'] == 'methods'):
        cont_methods +=1
    elif(d['qualifier'] == 'diagnosis'):
        cont_diagnosis +=1
    elif(d['qualifier'] == 'pharmacology'):
        cont_pharmacology +=1
    else:
        cont_other +=1
        
print("Definicoes - anatomy total: "+str(cont_anatomy_total))
print("Definicoes - methods total: "+str(cont_methods_total))
print("Definicoes - diagnosis total: "+str(cont_diagnosis_total))
print("Definicoes - pharmacology total: "+str(cont_pharmacology_total))
print("Definicoes - outros total: "+str(cont_other))"""

#Definicoes por categoria

for t in dictValida:
    if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'anatomy & histology'):
        cont_anatomy +=1
    if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'methods'):
        cont_methods +=1
    if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'diagnosis'):
        cont_diagnosis +=1
    if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'pharmacology'):
        cont_pharmacology +=1
    if(dictMesh[dictValida[t]['ID']]['qualifier'] == '#'):
        cont_other +=1

print("\n\nDefinicoes avaliadas - anatomy: "+str(cont_anatomy))
print("Definicoes avaliadas - methods: "+str(cont_methods))
print("Definicoes avaliadas - diagnosis: "+str(cont_diagnosis))
print("Definicoes avaliadas - pharmacology: "+str(cont_pharmacology))
print("Definicoes avaliadas - outros: "+str(cont_other))

        
#Termos originais

for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if not('<i>' in t):
            cont_originais +=1

print("\nTermos originais: "+str(cont_originais))

import gzip, pickle
with gzip.open("dictValida.dict.gz",'rb') as fd:
    dictValida = pickle.load(fd)
    fd.close()

#Termos semelhantes certos    

for t in dictValida:
    cont_avaliados += 1

print("Termos semelhantes avaliados: "+str(cont_avaliados))   

for t in dictValida:
    if(dictValida[t]['target'] == '1'):
        cont_certo_valida +=1

print("Termos semelhantes certos: "+str(cont_certo_valida))   

for t in dictValida:
    if(dictValida[t]['target'] == '1'):
        if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'anatomy & histology'):
            cont_certo_anatomy +=1
        if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'methods'):
            cont_certo_methods +=1
        if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'diagnosis'):
            cont_certo_diagnosis +=1
        if(dictMesh[dictValida[t]['ID']]['qualifier'] == 'pharmacology'):
            cont_certo_pharmacology +=1
        if(dictMesh[dictValida[t]['ID']]['qualifier'] == '#'):
            cont_certo_other +=1

        
print("Termos semelhantes certos - anatomy: "+str(cont_certo_anatomy))
print("Termos semelhantes certos - methods: "+str(cont_certo_methods))
print("Termos semelhantes certos - diagnosis: "+str(cont_certo_diagnosis))
print("Termos semelhantes certos - pharmacology: "+str(cont_certo_pharmacology))
print("Termos semelhantes certos - outros: "+str(cont_certo_other))

porcent_anatomy = (cont_certo_anatomy*100)/cont_anatomy
porcent_methods = (cont_certo_methods*100)/cont_methods
porcent_diagnosis = (cont_certo_diagnosis*100)/cont_diagnosis
porcent_pharmacology = (cont_certo_pharmacology*100)/cont_pharmacology
porcent_other = (cont_certo_other*100)/cont_other
porcent_total = (cont_certo_valida*100)/cont_avaliados

acuracia = (cont_certo_valida)/cont_avaliados

print("\nAcuracia: "+str(acuracia ))
print("\nPorcentagem termos semelhantes certos - totais: "+str(porcent_total ))
print("Porcentagem termos semelhantes certos - anatomy: "+str(porcent_anatomy ))
print("Porcentagem termos semelhantes certos - methods: "+str(porcent_methods))
print("Porcentagem termos semelhantes certos - diagnosis: "+str(porcent_diagnosis))
print("Porcentagem termos semelhantes certos - pharmacology: "+str(porcent_pharmacology))
print("Porcentagem termos semelhantes certos - outros: "+str(porcent_other))

Termos novos adicionados: 13934

Definicoes enriquecidas: 3977
Definicoes: 28939


Definicoes avaliadas - anatomy: 76
Definicoes avaliadas - methods: 155
Definicoes avaliadas - diagnosis: 153
Definicoes avaliadas - pharmacology: 209
Definicoes avaliadas - outros: 201

Termos originais: 336721
Termos semelhantes avaliados: 794
Termos semelhantes certos: 111
Termos semelhantes certos - anatomy: 15
Termos semelhantes certos - methods: 16
Termos semelhantes certos - diagnosis: 33
Termos semelhantes certos - pharmacology: 13
Termos semelhantes certos - outros: 34

Acuracia: 0.1397984886649874

Porcentagem termos semelhantes certos - totais: 13.97984886649874
Porcentagem termos semelhantes certos - anatomy: 19.736842105263158
Porcentagem termos semelhantes certos - methods: 10.32258064516129
Porcentagem termos semelhantes certos - diagnosis: 21.568627450980394
Porcentagem termos semelhantes certos - pharmacology: 6.220095693779904
Porcentagem termos semelhantes certos - outros: 16.9154228855

In [ ]:
#Primeiro

Termos novos adicionados: 6186

Definicoes enriquecidas: 1440
Definicoes: 28939


Definicoes avaliadas - anatomy: 38
Definicoes avaliadas - methods: 70
Definicoes avaliadas - diagnosis: 60
Definicoes avaliadas - pharmacology: 38
Definicoes avaliadas - outros: 96

Termos originais: 336721
Termos semelhantes avaliados: 302
Termos semelhantes certos: 156
Termos semelhantes certos - anatomy: 17
Termos semelhantes certos - methods: 41
Termos semelhantes certos - diagnosis: 39
Termos semelhantes certos - pharmacology: 17
Termos semelhantes certos - outros: 42

Porcentagem termos semelhantes certos - totais: 51.65562913907285
Porcentagem termos semelhantes certos - anatomy: 44.73684210526316
Porcentagem termos semelhantes certos - methods: 58.57142857142857
Porcentagem termos semelhantes certos - diagnosis: 65.0
Porcentagem termos semelhantes certos - pharmacology: 44.73684210526316
Porcentagem termos semelhantes certos - outros: 43.75
